In [19]:
lines = [
b']\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x04\x00\xc7\x0c',
b'\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x04\x00\xc7\x0c',
b'h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x04\x00\xc7\x0c',
###
b'B\x9c-$#\x11\x00\x00\x00(\xec$\x04\x00\xc7\x0c',
b"\x04\xbe! #\x03\x00\x00\x00H\x08'\x04\x00\xc7\x0c"
]
print(lines)
# s = lines[0]
# print(s)
# print(len(s))

[b']\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x04\x00\xc7\x0c', b'\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x04\x00\xc7\x0c', b'h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x04\x00\xc7\x0c', b'B\x9c-$#\x11\x00\x00\x00(\xec$\x04\x00\xc7\x0c', b"\x04\xbe! #\x03\x00\x00\x00H\x08'\x04\x00\xc7\x0c"]


In [150]:
def parse_glonass_line(line):
    words = []
    for i in range(0, len(line), 4):
        words.append(line[i:i+4])
        # words.append([line[i+2], line[i+1], line[i], line[i+3]])
    W1, W2, W3, W4 = [int.from_bytes(word) for word in words]
    # w_t = W3
    # W3 = W1
    # W1 = w_t
    idle = W1 >> 31
    StringN = (W1 >> 27) & 0xF
    sf1Data = W1 & ((1 << 28) - 1)
    
    sf2Data = W2
    
    sf3Pad = W3 & 0x07FF
    Haming = (W3 >> 11) & 0xFF
    sf3Data = W3 & 0x1FFF
   
    sf_sf = (W4 >> 16) & 0xFFFF
    sf_f = (W4) & 0xFF
    sf4Pad = (W4 >> 8) & 0xFF
    
    Data = ((((((StringN << 27) + sf1Data << 32) + sf2Data) << 13) +  sf3Data) << 8) + Haming
    # for  x in [W1, W2, W3, W4]:
    #     print(f'{x:032b}')
    return idle, StringN, Data, Haming, sf_sf, sf_f, sf3Pad, sf4Pad
    

In [147]:
for line in lines:
    idle, StringN, Data, Haming, sf_sf, sf_f, sf3Pad, sf4Pad = parse_glonass_line(line)
    print(idle, StringN, Haming, sf_sf, sf_f, sf3Pad, sf4Pad)
    print(Data)
    print(f'{Data:085b}')
    print()


00000000101101000101110100011100
10110100100101100001111001100001
01111000011110000000000010010001
11000111000000000000010000001100
0 0 0 50944 12 145 4
106467947819822152192256
0000000010110100010111010001110010110100100101100001111001100001000001001000100000000

00000001001100000000001000011110
11101000000101101000100100001100
10101011001010000000000001100010
11000111000000000000010000001100
0 0 0 50944 12 98 4
179454816416914145829376
0000000100110000000000100001111011101000000101101000100100001100000000110001000000000

00000000101100000110100000011100
11000100110101011110111111110001
01100101101010000000000000101110
11000111000000000000010000001100
0 0 0 50944 12 46 4
104132129423209795694080
0000000010110000011010000001110011000100110101011110111111110001000000010111000000000

00101101100111000100001000100100
00000000000100010010001100000000
11101100001010000000000000100100
11000111000000000000010000001100
0 5 0 50944 12 36 4
8101372211263012891272192
00110101100111000100001000100

In [36]:
import math

math.log2(211753532287585863294843900395520)

107.38408507193314

In [24]:



def func(s):
    words = []
    for i in range(0, len(s), 4):
        words.append(s[i:i+4])
    print(words)
    bits = []
    for word in words: 
        bits.append(f'{int.from_bytes(word):032b}')
    sf1, sf2, sf3, sf4 = bits
    print(sf1, sf2, sf3, sf4, sep='\n')
    
    W1 = int.from_bytes(words[0])
    idle = W1 >> 31
    StringN = (W1 >> 27) & 0xFF
    sf1Data = W1 & ((1 << 28) - 1)
    print(idle, StringN, sf1Data)
    
    W2 = int.from_bytes(words[1])
    sf2Data = W2
    print(sf2Data)
    
    W3 = int.from_bytes(words[2])
    sf3Pad = W3 & 0x07FF
    Haming = (W3 >> 11) & 0xFF
    sf3Data = W3 & 0x1FFF
    print(sf3Data, Haming, sf3Pad)
    
    
    W4 = int.from_bytes(words[3])
    sfN = (W4 >> 16) & 0xFFFF
    sfF = (W4) & 0xFF
    sf4Pad = (W4 >> 8) & 0xFF
    print(sfN, sf4Pad, sfF)

for line in lines:
    print(line)
    func(line)
    print('\n\n')

b']\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x04\x00\xc7\x0c'
[b']\xb4\x00\x1c', b'\x1e\x96\xb4a', b'\x00xx\x91', b'\x04\x00\xc7\x0c']
01011101101101000000000000011100
00011110100101101011010001100001
00000000011110000111100010010001
00000100000000001100011100001100
0 11 229900316
513193057
6289 15 145
1024 199 12



b'\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x04\x00\xc7\x0c'
[b'\x020\x01\x1e', b'\x89\x16\xe8\x0c', b'\x00(\xabb', b'\x04\x00\xc7\x0c']
00000010001100000000000100011110
10001001000101101110100000001100
00000000001010001010101101100010
00000100000000001100011100001100
0 0 36700446
2299979788
2914 21 866
1024 199 12



b'h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x04\x00\xc7\x0c'
[b'h\xb0\x00\x1c', b'\xef\xd5\xc4\xf1', b'\x00\xa8e.', b'\x04\x00\xc7\x0c']
01101000101100000000000000011100
11101111110101011100010011110001
00000000101010000110010100101110
00000100000000001100011100001100
0 13 145752092
4023764209
1326 12 1326
1024 199 12



b'B\x9c-$#\x11\x00\x00\x00(\xec$\x04\x00\xc7\x0c

In [14]:
(0xFF+1) >> 8


1

In [88]:
data = []
with open('raw.log', 'rb') as raw:
    data = raw.readlines()
D = data[-10000:]

In [111]:
X = []
f = open('sfrbx.txt', 'w')
for i, d in enumerate(D):
    if b'SFRBX' in d:
        X.append(d)
        print(D[i-1].decode('utf-8'), end=',', file=f)
f.close()

In [112]:
X = [
    b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xcf\xcd\x04\'7<\xe6\x11z<\x01$\n\xc0R\xbf,\x1fC\xa8\xfcv\xa1+\x95\xb5\x84*\xbb\x81\xfb\xae\x93\x13\xc0=A&'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc5\x0cG;'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc5\x0c6\xda'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xd9\x88\xac:I\xbf\xb4\xa4\x00\x00\xc3\xb7\x02\x00\xc5\x0cU\x03'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xd9\x88\xac:I\xbf\xb4\xa4\x00\x00\xc3\xb7\x02\x00\xc5\x0cD\xa2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xde.<E\x9c\xaf\xb05\x008\xf86\x02\x00\xc5\x0cW\xe2'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"3\xe9\x04\'L\x004\x11z\x9fA\x15\xaa\xee`\x99\xae\xb08\x9e\'{\xba\xae\xa4q\x9d0`\xdd?\x00\xa3\xc7\xee4f<'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00p\xd9\xc9N\xec\xbf4 \x00\x80S\xb0\x02\x00\xc5\x0c\x16\x89'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc2\x04@U@\xb0\x10[\x00\xa8nF\x02\x00\xc5\x0cF\xa9'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc5\x0c\x88\x9f'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc5\x0c\x99\x00'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\'\n\x05\')N\x800\xf5\xc2\xd1\x8cJS\xe4\xbc @O\x80\x84E4\ny\xa8\x18\x03\xfeOX\x83\x1f\x9fq\x9d\xa1>'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xa9iLel6p\xbe\x00P~?\x02\x00\xc5\x0c\xd4^'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc5\x0c\x1d`'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc5\x0c.\xc1'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc1ePu\xccv\x80\xeb\x008\xc25\x02\x00\xc5\x0c\xfb\xdd'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc1ePu\xccv\x80\xeb\x008\xc25\x02\x00\xc5\x0c\xea|'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xaf+\x05\'s\xab\xf9?\xc1i\x0b\x0f\xf5\xc9\xf7\xbf\x0e\x17-\x8ai\x8fX\x88i1\xb7\xab\xd7c\xea\xbfhP\x830xV'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc5\x0cI('
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc5\x0c8\xc7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xb4@\x08\x1ei\x18\x94\x00\xe0\x07\xe7\x03\x00\xc5\x0cu\x13'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x1e\xb4@\x08\x0badn\x00\x08o\x1f\x03\x00\xc5\x0c\x12h'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x03\x00\xc5\x0c#\xe5'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x13L\x05\'w\xb7\x8e\x17\xa0\xd0>$(@O?\x17\x16C(1\xe9\xa0+\xcb\xb9\x10\xa7].g-k\x14@\xb4\xd6e'
,b"\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x03\x00\xc5\x0c\x1d'"
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\xb1\x01\x1c\\\xcbDx\x00H\xae\xf2\x03\x00\xc5\x0cB\n'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\x1bh$#\x15\x00\x00\x00\xa0,%\x03\x00\xc5\x0c\x887'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x03\x00\xc5\x0c\xc6\x11'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x03\x00\xc5\x0cj4'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x03\x00\xc5\x0c\xb7A'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x9bm\x05\'\xd3{\x13\x12\xc8\xaa\x01$+\x80W?\xf2:C(P\xef\x9a\x80\x87n\xb9\x03\x0fw\xfe\x0bt @\t\xf2\xb5'
,b'\xb5b\x02\x130\x00\x00\x19\x00\x00\n\x01\x02\x00\x03\x13\xc1"\x9bm\x05\'\xd3{\x13\x12\x9b\xaa\x01$+\x80W?\xf2:C(P\xef\x9a\x80\x87n\xb9\x03\xf2v\xfe\x0b\xf8 @\x89\xa9\x08'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe0\x0b\\5\x80\x8f\x00x\x008M\x15\x03\x00\xc5\x0c\xd2\r'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xe0\x0b\\5\x80\x8f\x00x\x008M\x15\x03\x00\xc5\x0c\xc1\xac'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc5\x0cue'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc5\x0cd\x04'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc5\x0cX\xe2'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc5\x0cG\x81'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xd3\x89\x05\'L\x004\x11z\x9fA\x15\xaa\xee`\x99\xae\xb08\x9e\'{\xba\xae\xa4q\x9d0`\xdd?\x00\xa3\xc7\xee4\xa7\xbe'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00C\x84\xb4M{\xc04\xe6\x00\xe8\xf6=\x03\x00\xc5\x0c\\N'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\x84\xb4M{\xc04\xe6\x00\xe8\xf6=\x03\x00\xc5\x0cm\xaf'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe3\x1ahU\xfcu@\x83\x00\xc0\xd5\x07\x03\x00\xc5\x0c\xbf\x9d'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xe3\x1ahU\xfcu@\x83\x00\xc0\xd5\x07\x03\x00\xc5\x0c\xae<'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xcd\x0c\xa4Y\xde\xc0\xb4\x82\x00\x90\xe7?\x03\x00\xc5\x0c\x84\n'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xcd\x0c\xa4Y\xde\xc0\xb4\x82\x00\x90\xe7?\x03\x00\xc5\x0c\x95k'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xbb\xaa\x05\')N\x800\xf5\xc2\xd1\x8cJS\xe4\xbc @O\x80\x84E4\ny\xa8\x18\x03\xfeOX\x83\x1f\x9fq\x9d\xd5\xee'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00M\x0ftex`\xe0+\x00\xd0\x15\x1a\x03\x00\xc5\x0c:\x02'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00M\x0ftex`\xe0+\x00\xd0\x15\x1a\x03\x00\xc5\x0cLp'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00M\x0ftex`\xe0+\x00\xd0\x15\x1a\x03\x00\xc5\x0c;\x0f'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x8cX\xc0ny\xc0\xb4\xb2\x00\xb8\x92=\x03\x00\xc5\x0c[\xfe'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x8cX\xc0ny\xc0\xb4\xb2\x00\xb8\x92=\x03\x00\xc5\x0cml'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x8cX\xc0ny\xc0\xb4\xb2\x00\xb8\x92=\x03\x00\xc5\x0c\\\x0b'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00j)|u\x14t`<\x00\x88>\x1e\x03\x00\xc5\x0c\xafo'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00j)|u\x14t`<\x00\x88>\x1e\x03\x00\xc5\x0c\xb0|'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00j)|u\x14t`<\x00\x88>\x1e\x03\x00\xc5\x0c\xc1\xdd'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xcb\xcb\x05\'s\xab\xf9?\xc1i\x0b\x0f\xf5\xc9\xf7\xbf\x0e\x17-\x8ai\x8fX\x88i1\xb7\xab\xd7c\xea\xbfhP\x8304&'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00)\x19<~t\xc0\xf4n\x00\xc8\x07:\x03\x00\xc5\x0c\xbe\xb4'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00)\x19<~t\xc0\xf4n\x00\xc8\x07:\x03\x00\xc5\x0c\xbf\xc1'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00)\x19<~t\xc0\xf4n\x00\xc8\x07:\x03\x00\xc5\x0c\xd0"'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00,\xbd@\x08\xc6B\xf0J\x00\xc8j\xfb\x04\x00\xc5\x0c\xc4\xd7'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x1e\xbc@\x08\x0badn\x00\xf0j\x1f\x04\x00\xc5\x0c\xfe\x1e'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xbc@\x08\x1ei\x18\x94\x00\x18\x02\xe7\x04\x00\xc5\x0c\xb1\xf9'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x01\x01\xc1\x10-\x8bt\xfb\x00\x90\xd4T\x04\x00\xc5\x0c\xd6\xa2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x04\x00\xc5\x0c$\xe9'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\x04\xc1\x102V\x04\xea\x00\x10\x19\xd3\x04\x00\xc5\x0c4\xe1'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x8f\xec\x05\'\x94\x84\xd5\x17\xc9\xe0\x01$\x1e\x00T\xbf\xadbC\xa8\x94*\xda\xab\xec9\x17\x07\xa6\xa3y%\x00\x02\x80\xb8\xb1L'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00]\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x04\x00\xc5\x0c\x9b)'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x04\x00\xc5\x0c\x1e+'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\xb1\x01\x1c\\\xcbDx\x00H\xae\xf2\x04\x00\xc5\x0cC\x0e'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00B\x9c-$#\x11\x00\x00\x00(\xec$\x04\x00\xc5\x0c\xbfj'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x04\x00\xc5\x0c\xc7\x15'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\x1bh$#\x15\x00\x00\x00\xa0,%\x04\x00\xc5\x0c\x89;'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x04\x00\xc5\x0c\xa6\xd7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x04\x00\xc5\x0c\xb8E'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x04\x00\xc5\x0ck8'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xe3\r\x06\'\x9b\xe8E\x12\x0c\xc2\x02$\x03\x00Z?\x07-C(\x97\xc7\xb5 V\xf6\x18\x14\x95\xbc\xe3\x90\x97\'\x00\x89B\xdf'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x88\x0f\x845h\x9c\xa0\xee\x00\xd0I\x1f\x04\x00\xc5\x0c>\xa6'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x88\x0f\x845h\x9c\xa0\xee\x00\xd0I\x1f\x04\x00\xc5\x0cP\x14'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x88\x0f\x845h\x9c\xa0\xee\x00\xd0I\x1f\x04\x00\xc5\x0c?\xb3'
,b"\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xc8\xa1.?Z\xc0\xf4\xbb\x008\xf1;\x04\x00\xc5\x0c'\xb7"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc8\xa1.?Z\xc0\xf4\xbb\x008\xf1;\x04\x00\xc5\x0c9%'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\xa1.?Z\xc0\xf4\xbb\x008\xf1;\x04\x00\xc5\x0c(\xc4'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\r6\x8cE\x0c\xda\x10&\x008\xd1\x1f\x04\x00\xc5\x0c|9'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\r8\x8cE\x0c\xda\x10&\x00\xd8\xd4\x1f\x04\x00\xc5\x0c"\xd6'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\r8\x8cE\x0c\xda\x10&\x00\xd8\xd4\x1f\x04\x00\xc5\x0c37'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x1f)\x06\'L\x004\x11z\x9fA\x15\xaa\xee`\x99\xae\xb08\x9e\'{\xba\xae\xa4q\x9d0`\xdd?\x00\xa3\xc7\xee4\x94p'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00;\x02\xd4O\x9e\xbf\xf4\xe4\x00(C\x86\x04\x00\xc5\x0c\xaa\x03'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00;\x02\xd4O\x9e\xbf\xf4\xe4\x00(C\x86\x04\x00\xc5\x0c\xab\x10'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00;\x02\xd4O\x9e\xbf\xf4\xe4\x00(C\x86\x04\x00\xc5\x0c\xbcq'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\rQ\x94Uh\xf2\xd0\xb8\x00\x10r \x04\x00\xc5\x0c\xef\xbb'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\rQ\x94Uh\xf2\xd0\xb8\x00\x10r \x04\x00\xc5\x0c\xf0\xc8'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\rQ\x94Uh\xf2\xd0\xb8\x00\x10r \x04\x00\xc5\x0c\x01)'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xce\xf2\x82\\\xc1\xbf\xf4e\x00P\xd1\x85\x04\x00\xc5\x0cA\x04'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xce\xf2\x82\\\xc1\xbf\xf4e\x00P\xd1\x85\x04\x00\xc5\x0cSr'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xce\xf2\x82\\\xc1\xbf\xf4e\x00P\xd1\x85\x04\x00\xc5\x0cB\x11'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x8fJ\x06\')N\x800\xf5\xc2\xd1\x8cJS\xe4\xbc @O\x80\x84E4\ny\xa8\x18\x03\xfeOX\x83\x1f\x9fq\x9dJ\xc0'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00$\x95\x98e4\n\xe1Q\x00\x903\x08\x04\x00\xc5\x0c\x15`'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00$\x97\x98e4\n\xe1Q\x00x2\x08\x04\x00\xc5\x0c\xff\xdd'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$\x97\x98e4\n\xe1Q\x00x2\x08\x04\x00\xc5\x0c\x10>'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00R\xdb\x1bk\x10\xc0\xf4q\x00h1\x84\x04\x00\xc5\x0c)\x1a'
,b"\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00R\xdb\x1bk\x10\xc0\xf4q\x00h1\x84\x04\x00\xc5\x0c*'"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00R\xdb\x1bk\x10\xc0\xf4q\x00h1\x84\x04\x00\xc5\x0c;\x88'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00H=\xa0u\x90\x0b\xb1,\x00@\x1a\x18\x04\x00\xc5\x0c\xa8p'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00H=\xa0u\x90\x0b\xb1,\x00@\x1a\x18\x04\x00\xc5\x0c\xa9}'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00H=\xa0u\x90\x0b\xb1,\x00@\x1a\x18\x04\x00\xc5\x0c\xba\xde'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x07k\x06\'s\xab\xf9?\xc1i\x0b\x0f\xf5\xc9\xf7\xbf\x0e\x17-\x8ai\x8fX\x88i1\xb7\xab\xd7c\xea\xbfhP\x830\x11\x98'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x0f\x9c>|\xb3\xbft\xd8\x00\x08 \x84\x04\x00\xc5\x0c\xf32'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x0f\x9c>|\xb3\xbft\xd8\x00\x08 \x84\x04\x00\xc5\x0c\x05\xa0'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x0f\x9c>|\xb3\xbft\xd8\x00\x08 \x84\x04\x00\xc5\x0c\xf4?'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00,\xc5@\x08\xc6B\xf0J\x000m\xfb\x05\x00\xc5\x0c8='
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x1e\xc4@\x08\x0badn\x00\x08m\x1f\x05\x00\xc5\x0c"T'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xc4@\x08\x1ei\x18\x94\x00\xe0\x05\xe7\x05\x00\xc5\x0c\x85\xff'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\x04\xc1\x102V\x04\xea\x00\x10\x19\xd3\x05\x00\xc5\x0c5\xe5'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x05\x00\xc5\x0c%\xed'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xf7\x8c\x06\'\xe4\xf9\x0f\x18\xb6O\x03$$@Z\xbf\x90RC(\xa8p\xdc z\xa4I*\x13.=\x93\x93\x0f\xc0+S\xe7'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\xb1\x01\x18\\\xcbDx\x000\xac\xf2\x05\x00\xc5\x0c&*'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x18\xef\xd5\xc4\xf1\x00\xd0g.\x05\x00\xc5\x0cE\x1f'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x94X\x05\x00H\x0e&\x05\x00\xc5\x0c\x89n'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\x1bh$#\x15\x00\x00\x00\xa0,%\x05\x00\xc5\x0c\x8a?'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x05\x00\xc5\x0c\xb9I'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x05\x00\xc5\x0cl<'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x7f\xad\x06\'\x12Z\x93\x12\xf7\xe4\x06\xa4-\x80R?\x8fIC\xa8\xa8\xa3P\x16l\x17\xc6\'me\x14 \xec\xed\xbf\xbd\xd2*'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00i?\xa85\xb4\xec\xd0\xe3\x00\x88\x9b\x16\x05\x00\xc5\x0c7\xd3'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00i?\xa85\xb4\xec\xd0\xe3\x00\x88\x9b\x16\x05\x00\xc5\x0cP\xf2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00i?\xa85\xb4\xec\xd0\xe3\x00\x88\x9b\x16\x05\x00\xc5\x0cH4'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xb1D\xb9;\xd2\xc0t\x97\x00\x98F\x86\x05\x00\xc5\x0c!\x1d'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xb1D\xb9;\xd2\xc0t\x97\x00\x98F\x86\x05\x00\xc5\x0c\x10\xbc'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00j.\xb4E,\x97\xb0\xcd\x00\xc0|\x1b\x05\x00\xc5\x0cN6'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00j.\xb4E,\x97\xb0\xcd\x00\xc0|\x1b\x05\x00\xc5\x0c_\x97'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"{\xc9\x06\'L\x004\x11z\x9fA\x15\xaa\xee`\x99\xae\xb08\x9e\'{\xba\xae\xa4q\x9d0`\xdd?\x00\xa3\xc7\xee4\x90@'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00_H\xbcM\xae\xbft6\x00\xe8\xd1\x01\x05\x00\xc5\x0c\xa7\xa1'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00_H\xbcM\xae\xbft6\x00\xe8\xd1\x01\x05\x00\xc5\x0c\xc0\xc0'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00_H\xbcM\xae\xbft6\x00\xe8\xd1\x01\x05\x00\xc5\x0c\xb8\x02'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00LB\xb8U(\xfd\xd0\xfc\x00 \x92\r\x05\x00\xc5\x0cq\x81'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00LB\xb8U(\xfd\xd0\xfc\x00 \x92\r\x05\x00\xc5\x0c\x8a\xa0'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00LB\xb8U(\xfd\xd0\xfc\x00 \x92\r\x05\x00\xc5\x0c\x82\xe2'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xf40&Z\xed\xc0t\x98\x00\xc06\x80\x05\x00\xc5\x0c\xf90'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xf40&Z\xed\xc0t\x98\x00\xc06\x80\x05\x00\xc5\x0c\x12O'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf40&Z\xed\xc0t\x98\x00\xc06\x80\x05\x00\xc5\x0c\n\x91'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\x13\xea\x06\')N\x800\xf5\xc2\xd1\x8cJS\xe4\xbc @O\x80\x84E4\ny\xa8\x18\x03\xfeOX\x83\x1f\x9fq\x9dn0'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00+"\xc0e\xf0\x9d\x90\xb9\x000\xb7\x14\x05\x00\xc5\x0cz\x08'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00+"\xc0e\xf0\x9d\x90\xb9\x000\xb7\x14\x05\x00\xc5\x0ci\xa7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x9b\xb9\x07o\xc5\xbft\x00\x00P%\x86\x05\x00\xc5\x0c\xf4\xcc'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x9b\xb9\x07o\xc5\xbft\x00\x00P%\x86\x05\x00\xc5\x0c\xe3k'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x9b\xb9\x07o\xc5\xbft\x00\x00P%\x86\x05\x00\xc5\x0c\xfc\x8a'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x08\x16\x11p\xc3)\\\xbc\x00H\x9d\xbb\x05\x00\xc5\x0czk'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x16\x11p\x00\x00\x00\x00\x00\xf0\x00\x00\x05\x00\xc5\x0c\xad\xb2'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\x16\x11p|f\tS\x00 \xa5\xb9\x05\x00\xc5\x0c\x96S'
,b'\xb5b\x02\x130\x00\x00\x1f\x00\x00\n\x0e\x02\x00\x03\x13\xc1"\xe7\x0b\x07\'\xf3\xab\xec?\xe1\xa0m\x1c\x81I\x1b\x80\x07\xe3}\xb7\x08\xc7R\x08\xd7\r\xa6\x16\xcc\xd5\xe9?\xe7hB\r\x1d\xbb'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x00\x00x\x00\x00\x00\x00\x00\x00\x00\x00\x05\x00\xc5\x0c\x9eR'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00W\x16Bx\xf58\xfe\x0b\x00\x90\xf2\x12\x05\x00\xc5\x0c0\xf0'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x9c\x85Az\xf7\xd9\x8b\xe0\x00\xa8E\x10\x05\x00\xc5\x0cK\xa1'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x1e\xcc@\x08\x0badn\x00\xf0h\x1f\x01\x00\xc6\x0c\n\xf8'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xcc@\x08\x1ei\x18\x94\x00\x18\x00\xe7\x01\x00\xc6\x0c\xbd\xd3'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x01\x00\xc6\x0c"\xdf'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\x04\xc1\x102V\x04\xea\x00\x10\x19\xd3\x01\x00\xc6\x0c2\xd7'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8b\x00\xc1\x10\x05\xaa|\xfa\x00\xb8y\xe4\x01\x00\xc6\x0c\xd2\xd7'
,b'\xb5b\x02\x130\x00\x00\x1f\x00\x00\n\x0e\x02\x00\x03\x13\xc1"\xa3,\x07\'s\x04B\x1e\x14\xc0\x914\xe1\xa4\x90\x08ZNK\xa3\x89@\xd4\x96]\xc15\x95\xc1`\x99\x80\xa3\xb2p\x92C\xcb'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\xb1\x01\x1c\\\xcbDx\x00H\xae\xf2\x01\x00\xc6\x0cA\x04'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x01\x00\xc6\x0cU\xad'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x01\x00\xc6\x0c\x1c!'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\x1bh$#\x15\x00\x00\x00\xa0,%\x01\x00\xc6\x0c\x871'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x03\x00\x00\x00H\x08'\x01\x00\xc6\x0c\xdc\xcb"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x01\x00\xc6\x0c\xc5\x0b'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x01\x00\xc6\x0ci.'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x01\x00\xc6\x0c\x82M'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x01\x00\xc6\x0c\xb6;'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x9f0\x0c5\x18\x96`4\x00\xe0\xc4\x0b\x01\x00\xc6\x0c5\xbb'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x9f0\x0c5\x18\x96`4\x00\xe0\xc4\x0b\x01\x00\xc6\x0c$Z'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x9f0\x0c5\x18\x96`4\x00\xe0\xc4\x0b\x01\x00\xc6\x0c=y'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xae\xd268x\xbet\x96\x00(\x10\xb0\x01\x00\xc6\x0cJ\x07'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xae\xd268x\xbet\x96\x00(\x10\xb0\x01\x00\xc6\x0c9\xa6'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xae\xd268x\xbet\x96\x00(\x10\xb0\x01\x00\xc6\x0cR\xc5'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x85\x0c\x10EX\xbd a\x00@\xe5A\x01\x00\xc6\x0c\x16='
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\x0c\x10EX\xbd a\x00@\xe5A\x01\x00\xc6\x0c\x05\xdc'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x85\x0c\x10EX\xbd a\x00@\xe5A\x01\x00\xc6\x0c\x1e\xfb'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00GS\xe9Nh\xbf\xb4\x1e\x00(\xc2\xb1\x01\x00\xc6\x0c\x99\xd2'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00GS\xe9Nh\xbf\xb4\x1e\x00(\xc2\xb1\x01\x00\xc6\x0c\x88q'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00GS\xe9Nh\xbf\xb4\x1e\x00(\xc2\xb1\x01\x00\xc6\x0c\xa1\x90'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00f6\x1cU\x00\xaf\xc0\xd3\x00\x10o@\x01\x00\xc6\x0c1I'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00f6\x1cU\x00\xaf\xc0\xd3\x00\x10o@\x01\x00\xc6\x0cJh'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00f6\x1cU\x00\xaf\xc0\xd3\x00\x10o@\x01\x00\xc6\x0cB\xaa'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xe33\xd0^[\xbf\xf4\\\x00\xa0T\xb0\x01\x00\xc6\x0cu\x84'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xe33\xd0^[\xbf\xf4\\\x00\xa0T\xb0\x01\x00\xc6\x0c\x8e\xa3'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe33\xd0^[\xbf\xf4\\\x00\xa0T\xb0\x01\x00\xc6\x0c\x86\xe5'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00J\x80$e\xb4\xca\xb0\xec\x00\x08K \x01\x00\xc6\x0c\x03I'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00J\x80$e\xb4\xca\xb0\xec\x00\x08K \x01\x00\xc6\x0c\x14\xaa'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00z\xacMns\xbftx\x00 c\xb0\x01\x00\xc6\x0cU\xa7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00z\xacMns\xbftx\x00 c\xb0\x01\x00\xc6\x0cf\x08'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xe0G,u\xdc\xcb \xa0\x00\x98\x07\x11\x01\x00\xc6\x0c\x02>'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xe0G,u\xdc\xcb \xa0\x00\x98\x07\x11\x01\x00\xc6\x0c\x1b]'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe0G,u\xdc\xcb \xa0\x00\x98\x07\x11\x01\x00\xc6\x0c\x13\x9f'
,b"\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x1e\xc8'\x7f\x80\xbf\xb4\x82\x00\xb0\x17\xb6\x01\x00\xc6\x0c\xa1\x10"
,b"\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x1e\xc8'\x7f\x80\xbf\xb4\x82\x00\xb0\x17\xb6\x01\x00\xc6\x0c\xb2q"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xd4@\x08\x1ei\x18\x94\x00\xe8\x03\xe7\x02\x00\xc6\x0c\x99\x11'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x02\x00\xc6\x0c#\xe3'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x02\x00\xc6\x0c\x1d%'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x02\x00\xc6\x0cV\xb1'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x02\x00\xc6\x0c\xc6\x0f'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x03\x00\x00\x00H\x08'\x02\x00\xc6\x0c\xdd\xcf"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x02\x00\xc6\x0c\xb7?'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"O\xed\x07\'\x8e\xc1\x11\x13S&\x03\xa4\n\x00V?\xda\x1fC\xa8\xaf\x11\x00\xb7R\x1d\xd8\x0e\xa8`\x84\xa7\xec\xfb\xff.\xf9\xcb'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc6\x0cH='
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc6\x0cP\xfb'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xd9\x88\xac:I\xbf\xb4\xa4\x00\x00\xc3\xb7\x02\x00\xc6\x0c^\xc3'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xd9\x88\xac:I\xbf\xb4\xa4\x00\x00\xc3\xb7\x02\x00\xc6\x0cV\x05'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xde.<E\x9c\xaf\xb05\x008\xf86\x02\x00\xc6\x0c`\xa2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xde.<E\x9c\xaf\xb05\x008\xf86\x02\x00\xc6\x0cX\xe4'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xfb\t\x08\'L\x004\x11z\x9fA\x15\xaa\xee`\x99\xae\xb08\x9e\'{\xba\xae\xa4q\x9d0`\xdd?\x00\xa3\xc7\xee4RD'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00p\xd9\xc9N\xec\xbf4 \x00\x80S\xb0\x02\x00\xc6\x0c\x17\x8b'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00p\xd9\xc9N\xec\xbf4 \x00\x80S\xb0\x02\x00\xc6\x0c\x1fI'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xc2\x04@U@\xb0\x10[\x00\xa8nF\x02\x00\xc6\x0cOi'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc2\x04@U@\xb0\x10[\x00\xa8nF\x02\x00\xc6\x0cG\xab'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc6\x0c\x89\xa1'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc6\x0c\xa2\xc0'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc6\x0c\x9a\x02'
,b'\xb5b\x02\x130\x00\x00\x15\x00\x00\n\x07\x02\x00\x03\x13\xc1"\x93*\x08\'\x8e\xb1B\x12\xba*\xda\x8cPN\n\x9f\x0c\x03n\x02\x1cp\xd4:C(u\x00\xd5\xa3X\x03L\x9fq\x9d;\x1e'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xa9iLel6p\xbe\x00P~?\x02\x00\xc6\x0c\xc4\xff'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xa9iLel6p\xbe\x00P~?\x02\x00\xc6\x0c\xdd\x1e'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xa9iLel6p\xbe\x00P~?\x02\x00\xc6\x0c\xd5`'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc6\x0c\x1eb'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc6\x0c7\x81'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc6\x0c/\xc3'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc1ePu\xccv\x80\xeb\x008\xc25\x02\x00\xc6\x0c\xeb~'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc1ePu\xccv\x80\xeb\x008\xc25\x02\x00\xc6\x0c\xfc\xdf'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xe3K\x08\'s\xab\xf9?\xc1i\x0b\x0f\xf5\xc9\xf7\xbf\x0e\x17-\x8ai\x8fX\x88i1\xb7\xab\xd7c\xea\xbfhP\x830\xcfl'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc6\x0c9\xc9'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc6\x0cR\xe8'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc6\x0cJ*'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x1e\xdc@\x08\x0badn\x00\xf8n\x1f\x03\x00\xc6\x0c*L'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xdc@\x08\x1ei\x18\x94\x00\x10\x06\xe7\x03\x00\xc6\x0c\xcd\xb7'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\x04\xc1\x102V\x04\xea\x00\x10\x19\xd3\x03\x00\xc6\x0c4\xdf'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8b\x00\xc1\x10\x05\xaa|\xfa\x00\xb8y\xe4\x03\x00\xc6\x0c\xd4\xdf'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x03\x00\xc6\x0c$\xe7'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"\xa7l\x08\'\x91 (\x1d\xb4 \x08\x82\x9d \x08\x02\xb4 \x08\x82\x9d \x08\x02\xb4 \x08\x82\x9d \x08\x02\xb4 \x08\x82kT'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\xb1\x01\x1c\\\xcbDx\x00H\xae\xf2\x03\x00\xc6\x0cC\x0c'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x03\x00\xc6\x0cW\xb5'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x03\x00\xc6\x0c\x1e)'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\x1bh$#\x15\x00\x00\x00\xa0,%\x03\x00\xc6\x0c\x899'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x03\x00\x00\x00H\x08'\x03\x00\xc6\x0c\xde\xd3"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x03\x00\xc6\x0c\xc7\x13'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x03\x00\xc6\x0c\x84U'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x03\x00\xc6\x0c\xb8C'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xe0\x0b\\5\x80\x8f\x00x\x008M\x15\x03\x00\xc6\x0c\xdb\xcd'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe0\x0b\\5\x80\x8f\x00x\x008M\x15\x03\x00\xc6\x0c\xd3\x0f'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc6\x0ce\x06'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc6\x0c~%'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc6\x0cvg'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc6\x0cH\x83'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc6\x0ca\xa2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc6\x0cY\xe4'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"g\xa9\x08\'L\x004\x11z\x9fA\x15\xaa\xee`\x99\xae\xb08\x9e\'{\xba\xae\xa4q\x9d0`\xdd?\x00\xa3\xc7\xee4^T'
,b'\xb5b\x02\x130\x00\x00\x15\x00\x00\n\x07\x02\x00\x03\x13\xc1"g\xa9\x08\'\x1f\x004\x11z\x9fA\x15\xaa\xee`\x99\xae\xb08\x9e\x89z\xba\xae\x9fq]\x92\xd2\xfa?\x00+e\x89\x83N9'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00C\x84\xb4M{\xc04\xe6\x00\xe8\xf6=\x03\x00\xc6\x0c]P'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00C\x84\xb4M{\xc04\xe6\x00\xe8\xf6=\x03\x00\xc6\x0cvo'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\x84\xb4M{\xc04\xe6\x00\xe8\xf6=\x03\x00\xc6\x0cn\xb1'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xe3\x1ahU\xfcu@\x83\x00\xc0\xd5\x07\x03\x00\xc6\x0c\xaf>'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xe3\x1ahU\xfcu@\x83\x00\xc0\xd5\x07\x03\x00\xc6\x0c\xc8]'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe3\x1ahU\xfcu@\x83\x00\xc0\xd5\x07\x03\x00\xc6\x0c\xc0\x9f'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xcd\x0c\xa4Y\xde\xc0\xb4\x82\x00\x90\xe7?\x03\x00\xc6\x0c\x85\x0c'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xcd\x0c\xa4Y\xde\xc0\xb4\x82\x00\x90\xe7?\x03\x00\xc6\x0c\x9e+'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xcd\x0c\xa4Y\xde\xc0\xb4\x82\x00\x90\xe7?\x03\x00\xc6\x0c\x96m'
,b'\xb5b\x02\x130\x00\x00\x15\x00\x00\n\x07\x02\x00\x03\x13\xc1"\xf7\xca\x08\'\x8e\xb1B\x12\xba*\xda\x8cPN\n\x9f\x0c\x03n\x02\x1cp\xd4:C(u\x00\xd5\xa3X\x03L\x9fq\x9d?\x0e'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00M\x0ftex`\xe0+\x00\xd0\x15\x1a\x03\x00\xc6\x0c<\x11'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00M\x0ftex`\xe0+\x00\xd0\x15\x1a\x03\x00\xc6\x0cU0'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00M\x0ftex`\xe0+\x00\xd0\x15\x1a\x03\x00\xc6\x0cMr'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x8cX\xc0ny\xc0\xb4\xb2\x00\xb8\x92=\x03\x00\xc6\x0c]\r'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8cX\xc0ny\xc0\xb4\xb2\x00\xb8\x92=\x03\x00\xc6\x0cv,'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x8cX\xc0ny\xc0\xb4\xb2\x00\xb8\x92=\x03\x00\xc6\x0cnn'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00j)|u\x14t`<\x00\x88>\x1e\x03\x00\xc6\x0c\xb1~'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00j)|u\x14t`<\x00\x88>\x1e\x03\x00\xc6\x0c\xca\x9d'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00j)|u\x14t`<\x00\x88>\x1e\x03\x00\xc6\x0c\xc2\xdf'
,b'\xb5b\x02\x130\x00\x00\x03\x00\x00\n\x0c\x02\x00\x03\x13\xc1"\x7f\xeb\x08\'O\x16\xf5?\x18m\x99\x19.\x8a\x01\x00#1\xe1\x88OK\x80\tGx\x83=&U\xea?\xdc:\xbc\x82\xd2\x8c'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00)\x19<~t\xc0\xf4n\x00\xc8\x07:\x03\x00\xc6\x0c\xd9\xe2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00)\x19<~t\xc0\xf4n\x00\xc8\x07:\x03\x00\xc6\x0c\xd1$'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xe4@\x08\x1ei\x18\x94\x00\xf0\x03\xe7\x04\x00\xc6\x0c\xb3A'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00{\xe1@\x08\x90\x03y\x0e\x00`\x7f\x01\x04\x00\xc6\x0c\xdd\xd6'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\x04\xc1\x102V\x04\xea\x00\x10\x19\xd3\x04\x00\xc6\x0c5\xe3'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8b\x00\xc1\x10\x05\xaa|\xfa\x00\xb8y\xe4\x04\x00\xc6\x0c\xd5\xe3'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x04\x00\xc6\x0c%\xeb'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\xb1\x01\x1c\\\xcbDx\x00H\xae\xf2\x04\x00\xc6\x0cD\x10'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x04\x00\xc6\x0cX\xb9'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x04\x00\xc6\x0c\x1f-'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\x1bh$#\x15\x00\x00\x00\xa0,%\x04\x00\xc6\x0c\x8a='
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x03\x00\x00\x00H\x08'\x04\x00\xc6\x0c\xdf\xd7"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x04\x00\xc6\x0c\xc8\x17'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x04\x00\xc6\x0cl:'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x04\x00\xc6\x0c\x85Y'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x04\x00\xc6\x0c\xb9G'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x88\x0f\x845h\x9c\xa0\xee\x00\xd0I\x1f\x04\x00\xc6\x0c@\xb5'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x88\x0f\x845h\x9c\xa0\xee\x00\xd0I\x1f\x04\x00\xc6\x0cY\xd4'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x88\x0f\x845h\x9c\xa0\xee\x00\xd0I\x1f\x04\x00\xc6\x0cQ\x16'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\xa1.?Z\xc0\xf4\xbb\x008\xf1;\x04\x00\xc6\x0c)\xc6'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xc8\xa1.?Z\xc0\xf4\xbb\x008\xf1;\x04\x00\xc6\x0cB\xe5'
,b"\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc8\xa1.?Z\xc0\xf4\xbb\x008\xf1;\x04\x00\xc6\x0c:'"
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\r8\x8cE\x0c\xda\x10&\x00\xd8\xd4\x1f\x04\x00\xc6\x0c#\xd8'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\r8\x8cE\x0c\xda\x10&\x00\xd8\xd4\x1f\x04\x00\xc6\x0c<\xf7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\r8\x8cE\x0c\xda\x10&\x00\xd8\xd4\x1f\x04\x00\xc6\x0c49'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00;\x02\xd4O\x9e\xbf\xf4\xe4\x00(C\x86\x04\x00\xc6\x0c\xac\x12'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00;\x02\xd4O\x9e\xbf\xf4\xe4\x00(C\x86\x04\x00\xc6\x0c\xc51'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00;\x02\xd4O\x9e\xbf\xf4\xe4\x00(C\x86\x04\x00\xc6\x0c\xbds'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\rQ\x94Uh\xf2\xd0\xb8\x00\x10r \x04\x00\xc6\x0c\n\xe9'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\rQ\x94Uh\xf2\xd0\xb8\x00\x10r \x04\x00\xc6\x0c\x02+'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xce\xf2\x82\\\xc1\xbf\xf4e\x00P\xd1\x85\x04\x00\xc6\x0c\\2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xce\xf2\x82\\\xc1\xbf\xf4e\x00P\xd1\x85\x04\x00\xc6\x0cTt'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00$\x97\x98e4\n\xe1Q\x00x2\x08\x04\x00\xc6\x0c\x00\xdf'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00$\x97\x98e4\n\xe1Q\x00x2\x08\x04\x00\xc6\x0c\x19\xfe'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$\x97\x98e4\n\xe1Q\x00x2\x08\x04\x00\xc6\x0c\x11@'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00R\xdb\x1bk\x10\xc0\xf4q\x00h1\x84\x04\x00\xc6\x0cDH'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00R\xdb\x1bk\x10\xc0\xf4q\x00h1\x84\x04\x00\xc6\x0c<\x8a'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00H=\xa0u\x90\x0b\xb1,\x00@\x1a\x18\x04\x00\xc6\x0c\xc3\x9e'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00H=\xa0u\x90\x0b\xb1,\x00@\x1a\x18\x04\x00\xc6\x0c\xbb\xe0'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x0f\x9c>|\xb3\xbft\xd8\x00\x08 \x84\x04\x00\xc6\x0c\xf5A'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x0f\x9c>|\xb3\xbft\xd8\x00\x08 \x84\x04\x00\xc6\x0c\x0e`'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x0f\x9c>|\xb3\xbft\xd8\x00\x08 \x84\x04\x00\xc6\x0c\x06\xa2'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x1e\xec@\x08\x0badn\x00\xe0n\x1f\x05\x00\xc6\x0c$\x9c'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00{\xe9@\x08\x90\x03y\x0e\x00\x98z\x01\x05\x00\xc6\x0c\x19\xbc'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xec@\x08\x1ei\x18\x94\x00\x08\x06\xe7\x05\x00\xc6\x0c\xd7w'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\xc8\x04\xc1\x102V\x04\xea\x00\x10\x19\xd3\x05\x00\xc6\x0c6\xe7'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8b\x00\xc1\x10\x05\xaa|\xfa\x00\xb8y\xe4\x05\x00\xc6\x0c\xd6\xe7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x05\x00\xc6\x0c&\xef'
,b"\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x85\xb1\x01\x18\\\xcbDx\x000\xac\xf2\x05\x00\xc6\x0c',"
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1a\x89\x16\xe8\x0c\x00P\xa9b\x05\x00\xc6\x0c{\x95'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x18\xef\xd5\xc4\xf1\x00\xd0g.\x05\x00\xc6\x0cF!'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x94\x1bh$#\x15\x00\x00\x00\xa0,%\x05\x00\xc6\x0c\x8bA'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x83y\x01\x00p\r'\x05\x00\xc6\x0c\x07T"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x94X\x05\x00H\x0e&\x05\x00\xc6\x0c\x8ap'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x05\x00\xc6\x0cm>'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x05\x00\xc6\x0c\x86]'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x05\x00\xc6\x0c\xbaK'
,b'\xb5b\x02\x13\x18\x00\x06\t\x00\x05\x04\t\x02\x00i?\xa85\xb4\xec\xd0\xe3\x00\x88\x9b\x16\x05\x00\xc6\x0c8\xd5'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00i?\xa85\xb4\xec\xd0\xe3\x00\x88\x9b\x16\x05\x00\xc6\x0cI6'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xb1D\xb9;\xd2\xc0t\x97\x00\x98F\x86\x05\x00\xc6\x0c*\xdd'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xb1D\xb9;\xd2\xc0t\x97\x00\x98F\x86\x05\x00\xc6\x0c"\x1f'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00j.\xb4E,\x97\xb0\xcd\x00\xc0|\x1b\x05\x00\xc6\x0chW'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00j.\xb4E,\x97\xb0\xcd\x00\xc0|\x1b\x05\x00\xc6\x0c`\x99'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00_H\xbcM\xae\xbft6\x00\xe8\xd1\x01\x05\x00\xc6\x0c\xc1\xc2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00_H\xbcM\xae\xbft6\x00\xe8\xd1\x01\x05\x00\xc6\x0c\xb9\x04'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00LB\xb8U(\xfd\xd0\xfc\x00 \x92\r\x05\x00\xc6\x0c\x8b\xa2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00LB\xb8U(\xfd\xd0\xfc\x00 \x92\r\x05\x00\xc6\x0c\x83\xe4'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xf40&Z\xed\xc0t\x98\x00\xc06\x80\x05\x00\xc6\x0c\x13Q'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf40&Z\xed\xc0t\x98\x00\xc06\x80\x05\x00\xc6\x0c\x0b\x93'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x9b+"\xc0e\xf0\x9d\x90\xb9\x000\xb7\x14\x05\x00\xc6\x0c\x16U'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x9b+"\xc0e\xf0\x9d\x90\xb9\x000\xb7\x14\x05\x00\xc6\x0c\x1e\x13'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x9b\xb9\x07o\xc5\xbft\x00\x00P%\x86\x05\x00\xc6\x0c\xfd\x8c'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x9b\xb9\x07o\xc5\xbft\x00\x00P%\x86\x05\x00\xc6\x0c\xf5\xce'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x08\x16\x11p\xc3)\\\xbc\x00H\x9d\xbb\x05\x00\xc6\x0c{m'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\x16\x11p|f\tS\x00 \xa5\xb9\x05\x00\xc6\x0c\x97U'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00W\x16Bx\xf58\xfe\x0b\x00\x90\xf2\x12\x05\x00\xc6\x0c\x17\xc6'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x9c\x85Az\xf7\xd9\x8b\xe0\x00\xa8E\x10\x05\x00\xc6\x0cL\xa3'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00W\x16Bx\xf58\xfe\x0b\x00\x90\xf2\x12\x05\x00\xc6\x0c1\xf2'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00,\xf5@\x08\xc6B\xf0J\x00(m\xfb\x01\x00\xc7\x0c^\xc9'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00{\xf1@\x08\x90\x03y\x0e\x00hy\x01\x01\x00\xc7\x0c\xed\xd0'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xf4@\x08\x1ei\x18\x94\x00\xf8\x05\xe7\x01\x00\xc7\x0c\xcbk'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x01\x01\xc1\x10-\x8bt\xfb\x00\x90\xd4T\x01\x00\xc7\x0c\xd5\x9a'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8b\x00\xc1\x10\x05\xaa|\xfa\x00\xb8y\xe4\x01\x00\xc7\x0c\xd3\xd9'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x01\x00\xc7\x0c#\xe1'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00]\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x01\x00\xc7\x0c\x9a!'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x01\x00\xc7\x0cV\xaf'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x01\x00\xc7\x0c\x1d#'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x01\x00\xc7\x0c\xc6\r'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x03\x00\x00\x00H\x08'\x01\x00\xc7\x0c\xdd\xcd"
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x01\x00\xc7\x0c\x83O'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x01\x00\xc7\x0c\xb7='
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x9f0\x0c5\x18\x96`4\x00\xe0\xc4\x0b\x01\x00\xc7\x0c$O'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x9f0\x0c5\x18\x96`4\x00\xe0\xc4\x0b\x01\x00\xc7\x0c>{'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x9f0\x0c5\x18\x96`4\x00\xe0\xc4\x0b\x01\x00\xc7\x0c6\xbd'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xae\xd268x\xbet\x96\x00(\x10\xb0\x01\x00\xc7\x0c9\x9b'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xae\xd268x\xbet\x96\x00(\x10\xb0\x01\x00\xc7\x0cS\xc7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xae\xd268x\xbet\x96\x00(\x10\xb0\x01\x00\xc7\x0cK\t'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x85\x0c\x10EX\xbd a\x00@\xe5A\x01\x00\xc7\x0c\x05\xd1'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x85\x0c\x10EX\xbd a\x00@\xe5A\x01\x00\xc7\x0c\x1f\xfd'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x85\x0c\x10EX\xbd a\x00@\xe5A\x01\x00\xc7\x0c\x17?'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00GS\xe9Nh\xbf\xb4\x1e\x00(\xc2\xb1\x01\x00\xc7\x0c\x88f'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00GS\xe9Nh\xbf\xb4\x1e\x00(\xc2\xb1\x01\x00\xc7\x0c\xa2\x92'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00GS\xe9Nh\xbf\xb4\x1e\x00(\xc2\xb1\x01\x00\xc7\x0c\x9a\xd4'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00f6\x1cU\x00\xaf\xc0\xd3\x00\x10o@\x01\x00\xc7\x0c1>'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00f6\x1cU\x00\xaf\xc0\xd3\x00\x10o@\x01\x00\xc7\x0cC\xac'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00f6\x1cU\x00\xaf\xc0\xd3\x00\x10o@\x01\x00\xc7\x0cKj'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xe33\xd0^[\xbf\xf4\\\x00\xa0T\xb0\x01\x00\xc7\x0c\x8f\xa5'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe33\xd0^[\xbf\xf4\\\x00\xa0T\xb0\x01\x00\xc7\x0c\x87\xe7'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00J\x80$e\xb4\xca\xb0\xec\x00\x08K \x01\x00\xc7\x0c\x03>'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00J\x80$e\xb4\xca\xb0\xec\x00\x08K \x01\x00\xc7\x0c\x1dj'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00J\x80$e\xb4\xca\xb0\xec\x00\x08K \x01\x00\xc7\x0c\x15\xac'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00z\xacMns\xbftx\x00 c\xb0\x01\x00\xc7\x0cU\x9c'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00z\xacMns\xbftx\x00 c\xb0\x01\x00\xc7\x0cg\n'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00z\xacMns\xbftx\x00 c\xb0\x01\x00\xc7\x0co\xc8'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xe0G,u\xdc\xcb \xa0\x00\x98\x07\x11\x01\x00\xc7\x0c\x023'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xe0G,u\xdc\xcb \xa0\x00\x98\x07\x11\x01\x00\xc7\x0c\x1c_'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe0G,u\xdc\xcb \xa0\x00\x98\x07\x11\x01\x00\xc7\x0c\x14\xa1'
,b"\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x1e\xc8'\x7f\x80\xbf\xb4\x82\x00\xb0\x17\xb6\x01\x00\xc7\x0c\xa1\x05"
,b"\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x1e\xc8'\x7f\x80\xbf\xb4\x82\x00\xb0\x17\xb6\x01\x00\xc7\x0c\xb3s"
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00,\xfd@\x08\xc6B\xf0J\x00\xd0h\xfb\x02\x00\xc7\x0c\n\xbf'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00{\xf9@\x08\x90\x03y\x0e\x00\x90|\x01\x02\x00\xc7\x0c!v'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\xfc@\x08\x1ei\x18\x94\x00\x00\x00\xe7\x02\x00\xc7\x0c\xd7\x01'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x01\x01\xc1\x10-\x8bt\xfb\x00\x90\xd4T\x02\x00\xc7\x0c\xd6\x9e'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8b\x00\xc1\x10\x05\xaa|\xfa\x00\xb8y\xe4\x02\x00\xc7\x0c\xd4\xdd'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x02\x00\xc7\x0c$\xe5'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00]\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x02\x00\xc7\x0c\x9b%'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x02\x00\xc7\x0cW\xb3'
,b"\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x02\x00\xc7\x0c\x1e'"
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00B\x9c-$#\x11\x00\x00\x00(\xec$\x02\x00\xc7\x0c\xbff'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x03\x00\x00\x00H\x08'\x02\x00\xc7\x0c\xde\xd1"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x02\x00\xc7\x0c\xc7\x11'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x02\x00\xc7\x0c\xa6\xd3'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x02\x00\xc7\x0c\x84S'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x02\x00\xc7\x0c\xb8A'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc7\x0c7\xd1'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc7\x0cQ\xfd'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc7\x0cI?'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xd9\x88\xac:I\xbf\xb4\xa4\x00\x00\xc3\xb7\x02\x00\xc7\x0cE\x99'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xd9\x88\xac:I\xbf\xb4\xa4\x00\x00\xc3\xb7\x02\x00\xc7\x0c_\xc5'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xd9\x88\xac:I\xbf\xb4\xa4\x00\x00\xc3\xb7\x02\x00\xc7\x0cW\x07'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xde,<E\x9c\xaf\xb05\x00\xd0\xf96\x02\x00\xc7\x0c\xde\x88'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xde.<E\x9c\xaf\xb05\x008\xf86\x02\x00\xc7\x0ca\xa4'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xde.<E\x9c\xaf\xb05\x008\xf86\x02\x00\xc7\x0cY\xe6'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00p\xd9\xc9N\xec\xbf4 \x00\x80S\xb0\x02\x00\xc7\x0c K'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00p\xd9\xc9N\xec\xbf4 \x00\x80S\xb0\x02\x00\xc7\x0c\x18\x8d'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xc2\x04@U@\xb0\x10[\x00\xa8nF\x02\x00\xc7\x0cPk'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc2\x04@U@\xb0\x10[\x00\xa8nF\x02\x00\xc7\x0cH\xad'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc7\x0c\x89\x96'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc7\x0c\xa3\xc2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x02\xc2b^z\xbf4\xb6\x00\xb0`\xae\x02\x00\xc7\x0c\x9b\x04'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xa9iLel6p\xbe\x00P~?\x02\x00\xc7\x0c\xc4\xf4'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xa9iLel6p\xbe\x00P~?\x02\x00\xc7\x0c\xde '
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xa9iLel6p\xbe\x00P~?\x02\x00\xc7\x0c\xd6b'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc7\x0c\x1eW'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc7\x0c8\x83'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00k\xf2loj\xc0t4\x00\xd0\xe5;\x02\x00\xc7\x0c0\xc5'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xc1ePu\xccv\x80\xeb\x008\xc25\x02\x00\xc7\x0c\xebs'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xc1ePu\xccv\x80\xeb\x008\xc25\x02\x00\xc7\x0c\x05\x9f'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xc1ePu\xccv\x80\xeb\x008\xc25\x02\x00\xc7\x0c\xfd\xe1'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc7\x0c9\xbe'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc7\x0cS\xea'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xf1\xe2\x06x\xbf\xc0t\xbe\x00@\x96=\x02\x00\xc7\x0cK,'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00{\x01A\x08\x90\x03y\x0e\x00`\x7f\x01\x03\x00\xc7\x0c\xfe\xc2'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00C\x04A\x08\x1ei\x18\x94\x00\xf0\x03\xe7\x03\x00\xc7\x0c\xd4-'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x01\x01\xc1\x10-\x8bt\xfb\x00\x90\xd4T\x03\x00\xc7\x0c\xd7\xa2'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x8b\x00\xc1\x10\x05\xaa|\xfa\x00\xb8y\xe4\x03\x00\xc7\x0c\xd5\xe1'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x83\x05\xc1\x10\x8eG$<\x00\xd8\xb0\xd8\x03\x00\xc7\x0c%\xe9'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00]\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x03\x00\xc7\x0c\x9c)'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x03\x00\xc7\x0cX\xb7'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x03\x00\xc7\x0c\x1f+'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00B\x9c-$#\x11\x00\x00\x00(\xec$\x03\x00\xc7\x0c\xc0j'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x04\xbe! #\x03\x00\x00\x00H\x08'\x03\x00\xc7\x0c\xdf\xd5"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00$l\x0e$#\x14\x00\x00\x00h\n&\x03\x00\xc7\x0c\xc8\x15'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x03\x00\xc7\x0c\xa7\xd7'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x00\x80\xc8(\x00"\x0f\x00\x00\xa0\x05\x00\x03\x00\xc7\x0c\x85W'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x00\x80\xc8(\x00"\x10\x00\x00\xe0\x00\x00\x03\x00\xc7\x0c\xb9E'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xe0\t\\5\x80\x8f\x00x\x00\xd0L\x15\x03\x00\xc7\x0cW\xa7'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xe0\x0b\\5\x80\x8f\x00x\x008M\x15\x03\x00\xc7\x0c\xdc\xcf'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xe0\x0b\\5\x80\x8f\x00x\x008M\x15\x03\x00\xc7\x0c\xd4\x11'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc7\x0ce\xfb'
,b"\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc7\x0c\x7f'"
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\x94\xeb\xc4>\xba\xc0\xb49\x00\x18\x06:\x03\x00\xc7\x0cwi'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc7\x0cHx'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc7\x0cb\xa4'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00\xfe\x16dE\xacz\xf0\x9e\x00\xe8\xa6$\x03\x00\xc7\x0cZ\xe6'
,b'\xb5b\x02\x130\x00\x00\x1c\x00\x00\n\x00\x02\x00\x03\x13\xc1"g,\x0c\'\xc1\xa1\x90\x1e\x02\x83\xef\x86\x8b\x97,\xaf\xa5\xfaq!\xf7\xde\xb7\xb6\x8dA8\x19\x81\xbf,\x9a\xc8Z\xa9\xa4:\x01'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00]\xb4\x00\x1c\x1e\x96\xb4a\x00xx\x91\x04\x00\xc7\x0c\x9d-'
,b'\xb5b\x02\x13\x18\x00\x06\x18\x00\t\x04\x0f\x02\x00\x020\x01\x1e\x89\x16\xe8\x0c\x00(\xabb\x04\x00\xc7\x0cY\xbb'
,b'\xb5b\x02\x13\x18\x00\x06\x10\x00\x06\x04\x12\x02\x00h\xb0\x00\x1c\xef\xd5\xc4\xf1\x00\xa8e.\x04\x00\xc7\x0c /'
,b'\xb5b\x02\x13\x18\x00\x06\x07\x00\x0c\x04\x03\x02\x00B\x9c-$#\x11\x00\x00\x00(\xec$\x04\x00\xc7\x0c\xc1n'
,
]

In [151]:
import struct

format = '<BBBBBBBB'
plb = {}
for msg in X:
    # print(msg)
    msg = msg[6:-2]
    gnssID, svId, _, freqId, numWords, chn, version, _ = struct.unpack(format, msg[:struct.calcsize(format)])
    # print(gnssID, svId, numWords, chn)
    if version == 0x02:
        chn = None
    msg = msg[struct.calcsize(format):]
    svId = int(svId)
    if gnssID == 6:
        if svId not in plb.keys():
            plb[svId] = []
        plb[svId].append((freqId, chn, parse_glonass_line(msg), msg))
        
    

In [129]:
print(plb.keys())

dict_keys([16, 9, 7, 24])


In [130]:
len(plb[9])


98

In [132]:
plb[9][0][-1]

b'\xf8Z05\xa8\x94\x10.\x00\xe0\xf1\x11\x02\x00\xc5\x0c6\xda'

In [ ]:
!!auto-py-to-exe